# Task 1: Word Embeddings (10 points)

This notebook will guide you through all steps necessary to train a word2vec model (Detailed description in the PDF).

## Imports

This code block is reserved for your imports. 

You are free to use the following packages: 

(List of packages)

In [1]:
# Imports
import pandas as pd
import string 
import numpy as np
import csv
import math
from torch import nn
import torch
from torch.utils.data import DataLoader,TensorDataset
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler

# 1.1 Get the data (0.5 points)

The Hindi portion HASOC corpus from [github.io](https://hasocfire.github.io/hasoc/2019/dataset.html) is already available in the repo, at data/hindi_hatespeech.tsv . Load it into a data structure of your choice. Then, split off a small part of the corpus as a development set (~100 data points).

If you are using Colab the first two lines will let you upload folders or files from your local file system.

In [2]:
#TODO: implement!

from google.colab import files
uploaded = files.upload()

In [3]:
data = pd.read_csv('hindi_hatespeech.tsv', sep='\t')
# data = data.head(100)

## 1.2 Data preparation (0.5 + 0.5 points)

* Prepare the data by removing everything that does not contain information. 
User names (starting with '@') and punctuation symbols clearly do not convey information, but we also want to get rid of so-called [stopwords](https://en.wikipedia.org/wiki/Stop_word), i. e. words that have little to no semantic content (and, but, yes, the...). Hindi stopwords can be found [here](https://github.com/stopwords-iso/stopwords-hi/blob/master/stopwords-hi.txt) Then, standardize the spelling by lowercasing all words.
Do this for the development section of the corpus for now.

* What about hashtags (starting with '#') and emojis? Should they be removed too? Justify your answer in the report, and explain how you accounted for this in your implementation.

In [4]:
# Stopword Read
stop_word_list = pd.read_csv('stopword.txt', sep='\s+', header=None)
stop_word_list = stop_word_list[0].tolist() 


In [5]:
# Removing Punctuations and other unnecessary signs
data['text'] = data['text'].str.replace('[{}]'.format(string.punctuation), '')
data['text'] = data['text'].str.replace('[{}]'.format('।'), '')
data['text'] = data['text'].str.replace('[{}]'.format('\n'), '')
data['text'] = data['text'].str.lower()

In [6]:
#TODO: implement!
# Removing stopwords
data['text'] = data['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in (stop_word_list)]))
len(data['text'])

4665

## 1.3 Build the vocabulary (0.5 + 0.5 points)

The input to the first layer of word2vec is an one-hot encoding of the current word. The output od the model is then compared to a numeric class label of the words within the size of the skip-gram window. Now

* Compile a list of all words in the development section of your corpus and save it in a variable ```V```.

In [8]:
#TODO: implement!
# Unique word Vocabulary
V = list(data['text'].str.split(' ', expand=True).stack().unique())
len(V)

20241

* Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

In [9]:
# One hot grabbing function
def word_to_one_hot(word, vocab):
  return vocab[word]


## 1.4 Subsampling (0.5 points)

The probability to keep a word in a context is given by:

$P_{keep}(w_i) = \Big(\sqrt{\frac{z(w_i)}{0.001}}+1\Big) \cdot \frac{0.001}{z(w_i)}$

Where $z(w_i)$ is the relative frequency of the word $w_i$ in the corpus. Now,
* Calculate word frequencies
* Define a function ```sampling_prob``` that takes a word (string) as input and returns the probabiliy to **keep** the word in a context.

In [10]:
# All words in the corpus
def all_word_corpus(data):
  all_word_list = []
  for value in data.str.split(" "):
    all_word_list.extend(value)

  return all_word_list

all_word_list = all_word_corpus(data["text"])


84625

In [11]:
#TODO: implement!

# Subsampling function
def sampling_prob(word, corpus):
  word_number_list = []
  word_freq_dict = {} # Each word frequency dictionary
  word_prob_dict = {}
  for each in word:
    zw_i = corpus.count(each)/ len(corpus)
    word_number_list.append(zw_i)
    word_freq_dict[each] = zw_i
    Pkeepw_i = (math.sqrt(zw_i/0.001) + 1) * (0.001/zw_i)
    word_prob_dict[each] = Pkeepw_i
  return word_freq_dict, word_prob_dict

In [12]:

word_frequency_dictionary, word_probability_dictionary = sampling_prob(V, all_word_list)


word_probability_dictionary: {'बांग्लादेश': 4.500366998769468, 'शानदार': 10.466840380114382, 'वापसी': 17.85971811308125, 'भारत': 0.941036074974523, '314': 93.82418474648706, 'रन': 6.564385088047257, 'रोका': 21.039000486144843, 'indvban': 5.058916251210868, 'cwc19': 3.370885777878733, 'सब': 0.9924365635534185, 'रंडी': 0.9630148187796911, 'नाच': 8.503228397070817, 'देखने': 6.2882502430724205, 'व्यस्त': 21.039000486144843, 'शांतीदूत': 93.82418474648706, 'होगा': 1.4036401232182503, 'रंडीरोना': 93.82418474648706, 'शुरू': 2.818980111767824, 'देंगे': 3.25188905093717, 'तुम': 0.9833686932240272, 'हरामियों': 10.466840380114382, 'बस': 1.8638572578599422, 'जूतों': 93.82418474648706, 'कमी': 7.588858686621763, 'शुक्र': 33.51948512304273, 'तुम्हारी': 2.63195387795486, 'लिंचिंग': 5.224836949297409, 'हिंदुओं': 3.9728031685625833, 'जागने': 93.82418474648706, 'देर': 9.707659228188032, 'सच': 2.740453484249694, 'होगी': 2.703110908341147, 'हरामी': 0.822461649845438, 'सुवर': 4.382061064470327, 'ड्रामा': 33.

In [14]:
def new_word_probability_dictionary(word_probability_dictionary, V, all_word_list, data):
  threshold = 1.9
  deleted_word_list = []
  for i in word_probability_dictionary:
    if word_probability_dictionary[i] <= threshold: 
      deleted_word_list.append(i)
  data = data.apply(lambda x: ' '.join([item for item in x.split() if item not in (deleted_word_list)]))
  V = list(data.str.split(' ', expand=True).stack().unique())
  all_word_list = all_word_corpus(data)
  word_frequency_dictionary, word_probability_dictionary = sampling_prob(V, all_word_list)
  return word_frequency_dictionary, word_probability_dictionary, V, data


In [15]:
word_frequency_dictionary, word_probability_dictionary, V, data= new_word_probability_dictionary(word_probability_dictionary, V, all_word_list, data["text"])

20150

# 1.5 Skip-Grams (1 point)

Now that you have the vocabulary and one-hot encodings at hand, you can start to do the actual work. The skip gram model requires training data of the shape ```(current_word, context)```, with ```context``` being the words before and/or after ```current_word``` within ```window_size```. 

* Have closer look on the original paper. If you feel to understand how skip-gram works, implement a function ```get_target_context``` that takes a sentence as input and [yield](https://docs.python.org/3.9/reference/simple_stmts.html#the-yield-statement)s a ```(current_word, context)```.

* Use your ```sampling_prob``` function to drop words from contexts as you sample them. 

In [16]:
#TODO: implement!
window_size = 4 # we define it later also.
def get_target_context(sentence, V):
  sentence_word_list = []
  
  for each in sentence.split(" "):
    sentence_word_list.append(each)
  
  for i in sentence_word_list:
    sentence_word_index = {}

    # For sentence size more than 4
    if len(sentence_word_list)>= window_size+1:
      # 
      if (len(sentence_word_list) - sentence_word_list.index(i)) > 2:

        if sentence_word_list.index(i)==0:
          sentence_word_index[i] = [V.index(sentence_word_list[1]),V.index(sentence_word_list[2]),V.index(sentence_word_list[3]),V.index(sentence_word_list[4])]

        elif sentence_word_list.index(i)==1:
          sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[2]),V.index(sentence_word_list[3]),V.index(sentence_word_list[4])]
        else:
          index_num = sentence_word_list.index(i)
          sentence_word_index[i] = [V.index(sentence_word_list[index_num-2]),V.index(sentence_word_list[index_num-1]),V.index(sentence_word_list[index_num+1]),V.index(sentence_word_list[index_num+2])]
      elif (len(sentence_word_list) - sentence_word_list.index(i)) == 2:
        sentence_word_index[i] = [V.index(sentence_word_list[len(sentence_word_list)-5]),V.index(sentence_word_list[len(sentence_word_list)-4]),V.index(sentence_word_list[len(sentence_word_list)-3]),V.index(sentence_word_list[len(sentence_word_list)-1])]
      else:
        sentence_word_index[i] = [V.index(sentence_word_list[len(sentence_word_list)-5]),V.index(sentence_word_list[len(sentence_word_list)-4]),V.index(sentence_word_list[len(sentence_word_list)-3]),V.index(sentence_word_list[len(sentence_word_list)-2])]
      yield sentence_word_index

    # For sentence size 4
    elif len(sentence_word_list) == window_size:
      if sentence_word_list.index(i) == 0:
      
        sentence_word_index[i] = [V.index(sentence_word_list[1]),V.index(sentence_word_list[2]),V.index(sentence_word_list[3]),V.index(sentence_word_list[1])]
     
      elif sentence_word_list.index(i)==1:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[2]),V.index(sentence_word_list[3]),V.index(sentence_word_list[0])]
      elif sentence_word_list.index(i)==2: 
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[1]),V.index(sentence_word_list[3]),V.index(sentence_word_list[0])]
      else:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[1]),V.index(sentence_word_list[2]),V.index(sentence_word_list[0])]
      yield sentence_word_index

    # For sentence size 3
    elif len(sentence_word_list) == window_size-1:
      if sentence_word_list.index(i) == 0:
        sentence_word_index[i] = [V.index(sentence_word_list[1]),V.index(sentence_word_list[2]),V.index(sentence_word_list[1]),V.index(sentence_word_list[2])]
      elif sentence_word_list.index(i)==1:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[2]),V.index(sentence_word_list[0]),V.index(sentence_word_list[2])]
      else:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[1]),V.index(sentence_word_list[0]),V.index(sentence_word_list[1])]
      yield sentence_word_index

    # For sentence size 2
    elif len(sentence_word_list) == window_size-2:
      if sentence_word_list.index(i) == 0:
        sentence_word_index[i] = [V.index(sentence_word_list[1]),V.index(sentence_word_list[1]),V.index(sentence_word_list[1]),V.index(sentence_word_list[1])]
      else:
        sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[0]),V.index(sentence_word_list[0]),V.index(sentence_word_list[0])]
      yield sentence_word_index

    # For sentence size 1
    elif len(sentence_word_list) == window_size-3:
      sentence_word_index[i] = [V.index(sentence_word_list[0]),V.index(sentence_word_list[0]),V.index(sentence_word_list[0]),V.index(sentence_word_list[0])]
      yield sentence_word_index

    else:
      pass
    

In [17]:
# All (current_word, context) dictionary
all_yield_wrods_context = {}
for each in data:
  gt = get_target_context(each, V)
  for x in gt:
    all_yield_wrods_context.update(x)

len(all_yield_wrods_context)

20150

# 1.6 Hyperparameters (0.5 points)

According to the word2vec paper, what would be a good choice for the following hyperparameters? 

* Embedding dimension
* Window size

Initialize them in a dictionary or as independent variables in the code block below. 

In [19]:
word_to_one_hot_list_for_tensor = []
context_list_for_tensor = []
df = pd.DataFrame(list(zip(V)), columns=['Vocalbulary'])
one_hot_vocabulary = pd.get_dummies(df.Vocalbulary)
for i in all_yield_wrods_context:
  word_to_one_hot_list_for_tensor.append(word_to_one_hot(i,one_hot_vocabulary))
  context_list_for_tensor.append(all_yield_wrods_context[i])

In [23]:
# Set hyperparameters
window_size = 4
embedding_size = 300

# More hyperparameters
learning_rate = 0.05
epochs = 50

In [28]:
# DataLoader
def dataloader_for_model(all_yield_wrods_context, V):
  word_to_one_hot_list_for_tensor = []
  context_list_for_tensor = []
  df = pd.DataFrame(list(zip(V)), columns=['Vocalbulary'])
  one_hot_vocabulary = pd.get_dummies(df.Vocalbulary)
  for i in all_yield_wrods_context:
    word_to_one_hot_list_for_tensor.append(word_to_one_hot(i,one_hot_vocabulary))
    context_list_for_tensor.append(all_yield_wrods_context[i])

  one_hot_tensor = torch.FloatTensor(word_to_one_hot_list_for_tensor)
  context_tensor = torch.FloatTensor(context_list_for_tensor)
  dataset = TensorDataset(one_hot_tensor,context_tensor)


  loader = DataLoader(
      dataset,
      batch_size=32,
      num_workers=0,
      shuffle=True
  )
  return loader

  

# 1.7 Pytorch Module (0.5 + 0.5 + 0.5 points)

Pytorch provides a wrapper for your fancy and super-complex models: [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The code block below contains a skeleton for such a wrapper. Now,

* Initialize the two weight matrices of word2vec as fields of the class.

* Override the ```forward``` method of this class. It should take a one-hot encoding as input, perform the matrix multiplications, and finally apply a log softmax on the output layer.

* Initialize the model and save its weights in a variable. The Pytorch documentation will tell you how to do that.

In [29]:
# Create model 

class Word2Vec(nn.Module):
  def __init__(self, length, embedding_size):
    super().__init__()
    self.input_layer = nn.Linear(in_features=length, out_features=embedding_size, bias=False)
    self.output = nn.Linear(in_features=embedding_size, out_features=length, bias=False)

  def forward(self, one_hot):
    x=self.input_layer(one_hot)
    x2 = self.output(x)

    out = F.log_softmax(x2)
    return out

torch.cuda.empty_cache()
model = Word2Vec(len(V), embedding_size)
if torch.cuda.is_available():
    model.cuda()


# 1.8 Loss function and optimizer (0.5 points)

Initialize variables with [optimizer](https://pytorch.org/docs/stable/optim.html#module-torch.optim) and loss function. You can take what is used in the word2vec paper, but you can use alternative optimizers/loss functions if you explain your choice in the report.

In [30]:
# Define optimizer and loss
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
criterion = criterion.to(device)

# 1.9 Training the model (3 points)

As everything is prepared, implement a training loop that performs several passes of the data set through the model. You are free to do this as you please, but your code should:

* Load the weights saved in 1.6 at the start of every execution of the code block
* Print the accumulated loss at least after every epoch (the accumulate loss should be reset after every epoch)
* Define a criterion for the training procedure to terminate if a certain loss value is reached. You can find the threshold by observing the loss for the development set.

You can play around with the number of epochs and the learning rate.

In [ ]:
# Define train procedure
def train(dataset):
  print("Training started")
  
  total_loss = 0
  stopping_loss_threshold = 1.0
  early_stop = False
  for epoch in range(epochs):
    for i, data in enumerate(dataset):
      inputs, labels = data
      inputs, labels = inputs.cuda(), labels.cuda()
      inputs, labels = inputs.to(device), labels.to(device, dtype=torch.int64)

      model.zero_grad()
      log_probs = model(inputs)
      loss = criterion(log_probs, torch.max(labels, 1)[1])
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

  
      if i % 100 == 99:    # print every 100 mini-batches
        print('epoch:%d, batch:%d, loss: %.3f' %
          (epoch + 1, i + 1, total_loss / 100))

        if (total_loss /100 ) <=stopping_loss_threshold:
          print("Early Stop.")
          print('epoch:%d, loss: %.3f' %
              (epoch + 1, total_loss / 100))
          early_stop = True
          break
        total_loss = 0.0

    if early_stop:
      print("Stopped")
      break
   
train(dataloader_for_model(all_yield_wrods_context, V))

print("Training finished")

# 1.10 Train on the full dataset (0.5 points)

Now, go back to 1.1 and remove the restriction on the number of sentences in your corpus. Then, reexecute code blocks 1.2, 1.3 and 1.6 (or those relevant if you created additional ones). 

* Then, retrain your model on the complete dataset.

* Now, the input weights of the model contain the desired word embeddings! Save them together with the corresponding vocabulary items (Pytorch provides a nice [functionality](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for this).

In [33]:
# Save the model
PATH = './model.pt'
torch.save(model.state_dict(), PATH)